In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS


In [3]:
llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,  # Azure OpenAI API Key
    api_version="2023-12-01-preview",  # API version (you can adjust it as needed)
    azure_endpoint=azure_openai_endpoint,  # Azure OpenAI endpoint
    deployment_name=deployment_name,  # Ensure deployment_name is the correct one you configured in Azure
    temperature=0  # Optional: Adjust the temperature for randomness in responses
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_35616\1637258887.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [24]:
#  Charger FAISS avec les métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)


def extract_flavor_from_query(query):
    # Utilisation d'une expression régulière pour extraire les saveurs de la phrase
    flavors = re.findall(r'\b(?:à la?|avec|et)\s*(\w+)', query.lower())
    # Join flavors into a single string separated by comma
    return ', '.join(flavors)

#  Fonction pour récupérer les documents contenant cette saveur
def search_by_flavor(query):
    #  Extraire la saveur demandée avec le LLM
    flavor = extract_flavor_from_query(query)
    print(f"Saveur détectée : {flavor}")  # Debugging

    # Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()

    # Filtrer uniquement ceux qui contiennent cette saveur
    filtered_docs = [doc for doc in all_docs if flavor in doc.metadata.get("saveur", "").lower()]

    if not filtered_docs:
        return f"Aucun e-liquide trouvé avec la saveur '{flavor}'."
    # Limiter aux 4 premiers documents
    filtered_docs = filtered_docs[:4]

    #  Afficher les résultats trouvés
    results = "\n\n".join([f" **Produit** : {doc.page_content}\n **Saveur** : {doc.metadata['saveur']}" for doc in filtered_docs])
    
    return f"Voici les e-liquides correspondant à la saveur '{flavor}':\n\n{results}"

je retravail # # 🔹 Fonction pour extraire la saveur demandée par l'utilisateur
def extract_flavor_from_query(query):
     prompt = f"Quelle est la saveur mentionnée dans cette phrase : '{query}' ? Donne seulement la saveur exacte, sans phrase supplémentaire."
     return llm.predict(prompt).strip().lower()  # Convertir en minuscule pour éviter les erreurs
 avec   

 def extract_flavor_from_query(query):
    # Utilisation d'une expression régulière pour extraire les saveurs de la phrase
    flavors = re.findall(r'\b(?:à la?|avec|et)\s*(\w+)', query.lower())
    # Join flavors into a single string separated by comma
    return ', '.join(flavors)
pour que Lime et fraise et pas que lime , fraise

In [25]:
# 🎯 Tester la fonction
query = "Je veux un e-liquide à la fraise et lime"
result = search_by_flavor(query)
print(result)

Saveur détectée : fraise, lime
Voici les e-liquides correspondant à la saveur 'fraise, lime':

 **Produit** : Découvrez une mono-saveur dédiée à la fraise des bois, dans un e-liquide fabriqué en France par PULP. Le produit est idéal pour débuter, déjàprêt à vaperet disponible en plusieurs taux de nicotine : 0, 3, 6 ou 12mg/ml. Le Fraise Sauvage est élaboré à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale à hauteur de 70PG et 30VG. Vous retrouverez ce e-liquide dans des flacons de 10ml en plastique PET avec embout fin. Ces fioles sont également équipées d'une sécurité enfant. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours avec le sens du détail. Ainsi, il vous présente aussi bien des e-liquides prêts à vaper, à booster, ou encore des concentrés DIY (Do It Yourself) aux saveurs finement reproduites et de diverses variétés. Nos con

que des produits qui ont les saveur demander 

In [68]:
import re

# Charger FAISS avec les métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

def extract_flavor_from_query(query):
    """Extrait les saveurs de la phrase utilisateur."""
    flavors = re.findall(r'\b(?:à la?|avec|et)\s*([\w-]+)', query.lower())  # Capture des mots après 'à la', 'avec', 'et'
    return set(flavors)  # Retourne un ensemble de saveurs pour éviter les doublons

def search_by_flavor(query):
    """Cherche les e-liquides contenant exactement les saveurs demandées."""
    # Extraire les saveurs demandées
    requested_flavors = extract_flavor_from_query(query)
    print(f"Saveurs détectées : {requested_flavors}")  # Debugging

    # Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()

    # Filtrer uniquement ceux qui contiennent exactement ces saveurs (ni plus, ni moins)
    filtered_docs = [
        doc for doc in all_docs 
        if set(doc.metadata.get("saveur", "").lower().split(", ")) == requested_flavors
    ]

    if not filtered_docs:
        return f"Aucun e-liquide trouvé avec **exactement** les saveurs '{', '.join(requested_flavors)}'."

    # Limiter aux 4 premiers résultats
    # filtered_docs = filtered_docs[:4]

    # Construire la réponse
    results = "\n\n".join([
        f"**Produit** : {doc.page_content}\n**Saveur** : {doc.metadata['saveur']}" 
        for doc in filtered_docs
    ])
    
    return f" Voici les e-liquides contenant **exactement** '{', '.join(requested_flavors)}':\n\n{results}"

#  Tester la fonction
query = "Je veux un e-liquide à la fraise et lime"
result = search_by_flavor(query)
print(result)


Saveurs détectées : {'fraise', 'lime'}
 Voici les e-liquides contenant **exactement** 'fraise, lime':

**Produit** : Découvrez une mono-saveur dédiée à la fraise des bois, dans un e-liquide fabriqué en France par PULP. Le produit est idéal pour débuter, déjàprêt à vaperet disponible en plusieurs taux de nicotine : 0, 3, 6 ou 12mg/ml. Le Fraise Sauvage est élaboré à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale à hauteur de 70PG et 30VG. Vous retrouverez ce e-liquide dans des flacons de 10ml en plastique PET avec embout fin. Ces fioles sont également équipées d'une sécurité enfant. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours avec le sens du détail. Ainsi, il vous présente aussi bien des e-liquides prêts à vaper, à booster, ou encore des concentrés DIY (Do It Yourself) aux saveurs finement reproduites et de diverses variétés. 

In [72]:

# Fonction pour extraire les saveurs de la requête utilisateur
def extract_flavor_from_query(query):
    """Extrait les saveurs de la phrase utilisateur."""
    flavors = re.findall(r'\b(?:à la?|avec|et)\s*([\w-]+)', query.lower())  # Capture des mots après 'à la', 'avec', 'et'
    return set(flavors)  # Retourne un ensemble de saveurs pour éviter les doublons

# Fonction pour effectuer la recherche par saveur dans FAISS
def search_by_flavor(query):
    """Cherche les e-liquides contenant exactement les saveurs demandées."""
    # Extraire les saveurs demandées
    requested_flavors = extract_flavor_from_query(query)
    print(f"Saveurs détectées : {requested_flavors}")  # Debugging

    # Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()

    # Filtrer uniquement ceux qui contiennent exactement ces saveurs (ni plus, ni moins)
    filtered_docs = [
        doc for doc in all_docs 
        if set(doc.metadata.get("saveur", "").lower().split(", ")) == requested_flavors
    ]

    if not filtered_docs:
        return f"Aucun e-liquide trouvé avec **exactement** les saveurs '{', '.join(requested_flavors)}'."

    # Limiter aux 4 premiers résultats
    filtered_docs = filtered_docs[:4]

    # Construire la réponse
    results = "\n\n".join([f"**Produit** : {doc.page_content}\n**Saveur** : {doc.metadata['saveur']}" 
                          for doc in filtered_docs])
    
    return f"Voici les e-liquides contenant **exactement** '{', '.join(requested_flavors)}':\n\n{results}"

# Intégration de FAISS en tant que retriever dans la chaîne RAG
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Définition du prompt pour la génération de réponse
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Pipeline de question-réponse
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Initialisation de la chaîne de question-réponse
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne
try:
    question = 'Propose moi un produit avec lime'
    response = qa_chain.invoke(question)
    print(response)

except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")


Je te propose le produit "Bloody Lime" de la gamme Fruizee par Eliquid France. C'est un e-liquide pour cigarette électronique aux saveurs de citron vert et de fruits rouges. Il est disponible en différents formats, y compris prêt à l'emploi et à booster, avec des options de nicotine variées. Ce produit est fabriqué en France et est connu pour ses saveurs fruitées et fraîches.


In [73]:
from langchain.schema import Document


In [74]:

# Étape 1: Charger les documents à partir du CSV avec gestion des métadonnées
try:
    loader = CSVLoader(file_path='../../RGBD/table_produits/produits.csv', encoding='utf-8')
    raw_documents = loader.load()
    print(f"Nombre de documents chargés : {len(raw_documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    raw_documents = []

# Étape 2: Transformer les documents pour inclure la colonne "saveur" dans les métadonnées
documents = []
for doc in raw_documents:
    metadata = doc.metadata  # Récupérer les métadonnées existantes
    if "saveur" in metadata:  # Vérifier si la colonne existe
        metadata["saveur"] = metadata["saveur"]  # Ajouter "saveur" aux métadonnées
    documents.append(Document(page_content=doc.page_content, metadata=metadata))

# Étape 3: Création et sauvegarde du vecteur FAISS avec métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store enregistré avec succès !")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Fonction pour extraire les saveurs de la requête utilisateur
def extract_flavor_from_query(query):
    """Extrait les saveurs de la phrase utilisateur."""
    flavors = re.findall(r'\b(?:à la?|avec|et)\s*([\w-]+)', query.lower())  # Capture des mots après 'à la', 'avec', 'et'
    return set(flavors)  # Retourne un ensemble de saveurs pour éviter les doublons

# Fonction pour effectuer la recherche par saveur dans FAISS
def search_by_flavor(query):
    """Cherche les e-liquides contenant exactement les saveurs demandées."""
    # Extraire les saveurs demandées
    requested_flavors = extract_flavor_from_query(query)
    print(f"Saveurs détectées : {requested_flavors}")  # Debugging

    # Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()

    # Filtrer uniquement ceux qui contiennent exactement ces saveurs (ni plus, ni moins)
    filtered_docs = [
        doc for doc in all_docs 
        if set(doc.metadata.get("saveur", "").lower().split(", ")) == requested_flavors
    ]

    if not filtered_docs:
        return f"Aucun e-liquide trouvé avec **exactement** les saveurs '{', '.join(requested_flavors)}'."

    # Limiter aux 4 premiers résultats
    filtered_docs = filtered_docs[:4]

    # Construire la réponse
    results = "\n\n".join([f"**Produit** : {doc.page_content}\n**Saveur** : {doc.metadata['saveur']}" 
                          for doc in filtered_docs])
    
    return f"Voici les e-liquides contenant **exactement** '{', '.join(requested_flavors)}':\n\n{results}"

# Intégration de FAISS en tant que retriever dans la chaîne RAG
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Définition du prompt pour la génération de réponse
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Fonction pour formater les documents récupérés dans un format lisible pour le modèle
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Initialisation de la chaîne de question-réponse
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne avec une requête utilisateur
try:
    question = 'Propose moi un produit avec lime'
    response = qa_chain.invoke(question)
    print(response)

except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")


Nombre de documents chargés : 1531
Vector store enregistré avec succès !
Je te propose le produit "Bloody Lime 50ml Fruizee" de Eliquid France. C'est un e-liquide aux saveurs fraîches de citron vert et de baies rouges, avec un ratio PG/VG de 30/70. Il est disponible en flacon de 60ml, surboosté en arômes, et peut être boosté en nicotine selon tes besoins. Voici le lien pour plus de détails : [Bloody Lime 50ml Fruizee](https://www.aromes-et-liquides.fr/e-liquide-fruizee/14130-bloody-lime-50-ml.html).


In [75]:
# Test de la chaîne avec une requête utilisateur
try:
    question = 'Propose moi un produit avec lime et fraise'
    response = qa_chain.invoke(question)
    print(response)

except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")

Je te propose le produit "Pack Dessert A&L" qui contient un e-liquide avec des saveurs de fraise et de lime. Ce pack comprend trois e-liquides de 10ml chacun, fabriqués en France, et est proposé par Arômes et Liquides.


In [82]:
# Fonction pour afficher les documents avec une saveur spécifiée
def display_docs_with_flavor(flavor):
    # Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()
    
    # Filtrer uniquement ceux qui contiennent la saveur demandée
    filtered_docs = [
        doc for doc in all_docs if "saveur" in doc.metadata and flavor.lower() in doc.metadata["saveur"].lower()
    ]
    
    # Affichage des documents filtrés
    if filtered_docs:
        for doc in filtered_docs:
            print(f"Produit : {doc.page_content}\nSaveur : {doc.metadata['saveur']}\n")
    else:
        print(f"Aucun document trouvé avec la saveur '{flavor}'.")

In [153]:
import pandas as pd

# Charger le CSV
csv_path = "../../RGBD/table_produits/produits.csv"   # Remplace avec le chemin réel si besoin
df = pd.read_csv(csv_path, encoding='utf-8')

# Vérifier que la colonne "description" existe bien
if "description" not in df.columns:
    raise ValueError("⚠️ La colonne 'description' n'existe pas dans le CSV ! Vérifie le nom des colonnes.")

# Transformer chaque ligne en objet Document avec métadonnées
documents = []
for index, row in df.iterrows():
    metadata = {"source": csv_path, "row": index}

    if "saveur" in row:
        metadata["saveur"] = str(row["saveur"])  # Ajouter la saveur dans les métadonnées

    documents.append(Document(page_content=str(row["description"]), metadata=metadata))

print(f"Nombre de documents préparés : {len(documents)}")

# Création et sauvegarde du FAISS vector store avec métadonnées
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store FAISS sauvegardé avec succès.")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
query = "Saveur cassis lime"
print("🚀 FAISS vector store chargé et prêt à l'emploi !")

Nombre de documents préparés : 1531
Vector store FAISS sauvegardé avec succès.
🚀 FAISS vector store chargé et prêt à l'emploi !


In [154]:

retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 281, 'saveur': 'cassis, lime, mangue'}, page_content="Le Cassis Mangue est un e-liquideà boosterde la collection Fruizee par Eliquid France, auxsaveurs fraîches de mangue et de cassis. Sa recette fabriquée en France se compose d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio 30PG/70VG. Vous pouvez au besoin ajouter un booster 10ml de nicotine (non inclus). Il vous est livré dans unflacon de 60mldoté d'un embout fin et d'un bouchon avec sécurité enfant. Qui est Eliquid France ? Créé en 2013 à Nieul sur Mer en Charente-Maritime, le laboratoire Eliquid France s'est fait une place de choix dans l'univers de la vape grâce à des liquides de qualité, exportant ses créations dans le monde entier. Toujours en recherche d'innovation, la marque française propose des recettes variées aux saveurs allant de simples à complexes, et se déclinant autant en arôme DIY qu'en e-liquide

In [89]:
# Fonction pour interroger le vecteur et récupérer la saveur associée
def search_for_flavor(query, vectorstore, retriever, llm):
    # Effectuer la recherche avec le modèle
    results = retriever.get_relevant_documents(query)

    # Afficher les saveurs des résultats
    for result in results:
        # Accéder à la saveur dans les métadonnées
        flavor = result.metadata.get("saveur", "Saveur non trouvée")
        print(f"Saveur trouvée : {flavor}")
        print(f"Description du produit : {result.page_content}\n")
        
# Exemple d'appel à la fonction avec une requête utilisateur
query = "Saveur cassis lime"  # Vous pouvez ajuster la requête comme nécessaire
search_for_flavor(query, vectorstore, retriever, llm)

Saveur trouvée : cassis, lime
Description du produit : Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide prêt-à-vaper, à booster, ou en concentré DIY ! La gamme Natural Le fa

C:\Users\antoa\AppData\Local\Temp\ipykernel_35616\41954493.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [132]:
import re

# Fonction pour extraire les saveurs de la requête et les rechercher dans les résultats
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors


# Fonction pour interroger le vecteur et récupérer les saveurs associées à la requête
def search_for_flavor(query):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    # Afficher uniquement les saveurs extraites
    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Retourner les saveurs extraites sous forme de chaîne, séparées par des virgules
    return ", ".join(flavors_in_query)

# Exemple d'appel à la fonction avec une requête utilisateur
query = "Je veux un e-liquide avec lime et cassis"  # Vous pouvez ajuster la requête comme nécessaire

# Utilisation de la fonction search_for_flavor dans retriever.invoke
flavors_in_query = search_for_flavor(query)



Saveurs extraites de la requête : ['cassis', 'lime']


In [133]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide

In [134]:
retriever.invoke(query, filter={"saveur": "lime"})

No relevant docs were retrieved using the relevance score threshold 0.1


[]

In [151]:
#  Charger FAISS avec les métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Fonction pour extraire les saveurs de la requête
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors


# Fonction pour interroger le vecteur et récupérer les saveurs associées à la requête
def search_for_flavor(query):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    # Afficher uniquement les saveurs extraites
    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Retourner les saveurs extraites sous forme de chaîne, séparées par des virgules
    return ", ".join(flavors_in_query)


# Exemple d'appel à la fonction avec une requête utilisateur
query = "Je veux un e-liquide avec cassis"

# Extraction des saveurs
flavors_in_query = search_for_flavor(query)



# Définition du prompt pour la génération de réponse
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

# Initialisation du prompt
prompt = ChatPromptTemplate.from_template(prompt_template)

# Formatage des documents de contexte
def format_docs(flavors):
    # Création du contexte avec les saveurs extraites
    return f"Les saveurs extraites sont : {flavors}"

# Pipeline de question-réponse
qa_chain = (
    {
        "context": retriever | format_docs,  # Utilisation du retriever pour obtenir des contextes
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()  # Extraction de la réponse
)

# Exécution de la chaîne de question-réponse et affichage du résultat
# Ici, tu passes les saveurs extraites en tant que contexte dans le modèle
response = qa_chain.invoke(query, filter={"saveur": flavors_in_query})
# Affichage du résultat
print(f"Réponse générée par le modèle : {response}")


Saveurs extraites de la requête : ['cassis']
Réponse générée par le modèle : Vous pouvez opter pour plusieurs e-liquides avec saveur cassis. Voici quelques options :

1. **Le Cassis de la collection Freeze par Liquideo** : Un e-liquide non nicotiné à booster, avec un goût de cassis, lime, baie, et baie noire. Il est fabriqué en France avec un ratio PG/VG de 50/50.

2. **Le Cassis de la collection Evolution par Liquideo** : Un e-liquide à booster avec des saveurs de cassis, lime, et menthol, également fabriqué en France avec un ratio PG/VG de 50/50.

3. **Le Cassis de la collection Natural par Curieux** : Propose une saveur fraîche de cassis, avec une composition de végétol (50%) et de glycérine végétale (50%).

4. **Le Framboise Cassis de la collection 1900 par Curieux** : Un e-liquide avec des saveurs de cassis et de framboise, avec un ratio PG/VG de 40/60.

Ces options vous permettent de choisir selon vos préférences de saveur et de composition.


In [145]:
flavors_in_query

'cassis, lime'

In [146]:
query

'Je veux un e-liquide avec lime et cassis'

In [ ]:
# Fonction pour extraire les saveurs de la requête
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors

In [ ]:
# Exemple d'appel à la fonction avec une requête utilisateur
query = "Je veux un e-liquide avec cassis"
extract_flavors_from_query(query)

In [152]:
# Requête de recherche
query = "Quel produit contient des saveurs de cassis ?"
results = retriever.get_relevant_documents(query)

# Extraction des métadonnées des résultats
for result in results:
    print(f"Contenu : {result.page_content}")
    print(f"Saveur : {result.metadata.get('saveur', 'Aucune saveur spécifiée')}") 

Contenu : Le Cassis Mangue est un e-liquideà boosterde la collection Fruizee par Eliquid France, auxsaveurs fraîches de mangue et de cassis. Sa recette fabriquée en France se compose d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio 30PG/70VG. Vous pouvez au besoin ajouter un booster 10ml de nicotine (non inclus). Il vous est livré dans unflacon de 60mldoté d'un embout fin et d'un bouchon avec sécurité enfant. Qui est Eliquid France ? Créé en 2013 à Nieul sur Mer en Charente-Maritime, le laboratoire Eliquid France s'est fait une place de choix dans l'univers de la vape grâce à des liquides de qualité, exportant ses créations dans le monde entier. Toujours en recherche d'innovation, la marque française propose des recettes variées aux saveurs allant de simples à complexes, et se déclinant autant en arôme DIY qu'en e-liquide prêt à vaper. La gamme Fruizee Collection ayant fait la réputation de la marque Eliquid France, elle mélange des saveurs f